<a href="https://colab.research.google.com/github/DrAlexSanz/Emojify/blob/master/Emojify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd "/content"
!rm -rf Emojify

!git clone https://github.com/DrAlexSanz/Emojify.git
  
%cd "/content/Emojify"

/content
Cloning into 'Emojify'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.
/content/Emojify


In [2]:
!pip install emoji
import numpy as np
import emoji
from emo_utils import *
import matplotlib.pyplot as plt

     |████████████████████████████████| 51kB 3.4MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=1b47bb8d40e27dc06593ee1d61235671795d48dc1fc0cd7e49630ebf963ab319
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
X_train, y_train = read_csv("Data/train_emoji.csv")
X_test, y_test = read_csv("Data/tesss.csv")

Now I have all the data read, train and test.

In [5]:
index = 13

print(X_train[index], label_to_emoji(y_train[index]))

my algorithm performs poorly 😞


This is a bit pessimistic. At least I'm not, but if you are, change the index and find a nicer one.

Since the labels are numerical, 0, 1, 2, etc. I have to convert them to one-hot encoding. I just happen to have a function for this. How convenient and how elegant.

In [0]:
Y_oh_train = convert_to_one_hot(y_train, C = 5)
Y_oh_test = convert_to_one_hot(y_test, C = 5)

In [15]:
# To test
index = 13
print(str(y_train[index]) + " converted to one-hot is " + str(Y_oh_train[index]))

3 converted to one-hot is [0. 0. 0. 1. 0.]


Now one thing I should do is to read the GloVe vectors. I have them somewhere in google drive.

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs("/content/drive/My Drive/Glove/glove.6B.50d.txt")

As a reminder:


*   word to index is a dictionary mapping, from word to the indices of the vocabulary.
*   index to word is the inverse mapping.
*   word to vec is the mapping of word towards vector.

If I want an index to vec mapping I need ot do it in two steps, but just that.



In [24]:
#Check this. Words are uncased!!

word = "cucumber"
index =325651

print("The index of " + word + " is: " + str(word_to_index[word]))
print("The word for index " + str(index) + " is " + str(index_to_word[index]))

The index of cucumber is: 113317
The word for index 325651 is serpent


Implement sentence to average. Take a sentence, split it and get the average of the 50D encoding. This will be useful later.

In [0]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    The function takes a sentence, makes it lowercase and split it and then convert the words to the vector repesentation
    Then the vectors are averaged
    
    Inputs: Sentence (string) and word_to_vec mapping
    
    Output: The average of the vector encoding    
    """
    
    # Split and lowercase
    
    words = [i.lower() for i in sentence.split()]
    
    #Initialize average
    avg = np.zeros((50,))
    
    for w in words:
        avg = avg + word_to_vec_map[w]
    
    avg = avg / len(words)
    
    return avg

In [0]:
cipote = sentence_to_avg("I am freezing my balls off", word_to_vec_map)
print(cipote)